<h1 align=center>Redes Adversariais Geradoras para Sintetizar Novos Dados</h1>

<p align=center><img src=https://advancedinstitute.ai/wp-content/uploads/2020/08/real_fake.png width=500></p>

Anteriomente, focamos em redes neurais recorrentes (RNN) para modelagem de sequências. Neste capítulo, exploraremos as redes generativas de adversários (GANs) e veremos sua aplicação na síntese de novas amostras de dados. As GANs são consideradas o avanço mais importante no aprendizado profundo, permitindo que os computadores gerem novos dados (como novas imagens).
Neste capítulo, abordaremos os seguintes tópicos:
• Apresentando modelos generativos para sintetizar novos dados
• Autoencoders, autoencoders variacionais (VAEs) e seu relacionamento com GANs
• Entendendo os blocos de construção das GANs
• Implementação de um modelo GAN simples para gerar dígitos manuscritos
• Noções básicas sobre convolução transposta e normalização de lote (BatchNorm ou BN)
• Melhorar GANs: GANs convolucionais profundos e GANs usando a distância de Wasserstein